In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models.word2vec import Word2Vec

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, plot_roc_curve, roc_auc_score, recall_score, accuracy_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
df = pd.read_csv('./temp_prec/city_info.csv')

In [ ]:
df.head()

In [ ]:
cities=df['Name'].unique()

In [ ]:
ids= df['ID'].unique()

In [ ]:
dic = dict(zip(ids, cities))

In [ ]:
df2 = pd.DataFrame()

for i , city in dic.items():
    df= pd.read_csv(f'./temp_prec/{i}.csv')
    df['City']=city
    df_total=pd.merge(df,df2,how='outer')

In [ ]:
df.shape

In [ ]:
df_total.head(20)

In [100]:
drought=pd.read_csv('./drought/dr.csv',decimal=".",sep=',')

In [101]:
drought.head()

,MapDate,Region,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID,ClimateDivisionsID
0,20211214,Central,0.00,0.00,0.00,"1,057.49","7,663.36","13,493.12",2021-12-14,2021-12-20,2,2404
1,20211214,Central,"2,607.81","4,372.41",0.00,0.00,0.00,0.00,2021-12-14,2021-12-20,2,4604
2,20211214,Cheyenne & Niobrara Drainage,0.00,"1,848.56","4,515.72",981.97,0.00,0.00,2021-12-14,2021-12-20,2,4807
3,20211214,East Central,115.62,"5,459.76",0.00,0.00,0.00,0.00,2021-12-14,2021-12-20,2,3206
4,20211214,Extreme Southern,0.00,0.00,0.00,"2,841.68","5,553.79","1,586.81",2021-12-14,2021-12-20,2,2604


In [102]:
drought.describe()

,MapDate,StatisticFormatID,ClimateDivisionsID
count,4.091220e+05,409122.0,409122.000000
mean,2.010548e+07,2.0,2578.904762
std,6.334369e+04,0.0,1439.873316
min,2.000010e+07,2.0,101.000000
25%,2.005063e+07,2.0,1305.000000
50%,2.010122e+07,2.0,2508.000000
75%,2.016062e+07,2.0,3805.000000
max,2.021121e+07,2.0,4913.000000


In [103]:
drought.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 409122 entries, 0 to 409121
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   MapDate             409122 non-null  int64 
 1   Region              409122 non-null  object
 2   None                409122 non-null  object
 3   D0                  409122 non-null  object
 4   D1                  409122 non-null  object
 5   D2                  409122 non-null  object
 6   D3                  409122 non-null  object
 7   D4                  409122 non-null  object
 8   ValidStart          409122 non-null  object
 9   ValidEnd            409122 non-null  object
 10  StatisticFormatID   409122 non-null  int64 
 11  ClimateDivisionsID  409122 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 37.5+ MB


In [104]:
drought['None'] = drought['None'].str.replace(",", "").astype(float)
drought['D0'] = drought['D0'].str.replace(",", "").astype(float)
drought['D1'] = drought['D1'].str.replace(",", "").astype(float)
drought['D2'] = drought['D2'].str.replace(",", "").astype(float)
drought['D3'] = drought['D3'].str.replace(",", "").astype(float)
drought['D4'] = drought['D4'].str.replace(",", "").astype(float)

In [105]:
drought['ValidStart']= pd.to_datetime(drought['ValidStart'])
drought['ValidEnd']= pd.to_datetime(drought['ValidEnd'])

In [106]:
drought.isnull().sum()

MapDate               0
Region                0
None                  0
D0                    0
D1                    0
D2                    0
D3                    0
D4                    0
ValidStart            0
ValidEnd              0
StatisticFormatID     0
ClimateDivisionsID    0
dtype: int64

In [117]:
drought=drought.drop(['ValidEnd','ValidStart'],axis=1)


In [118]:
drought.columns

Index(['MapDate', 'None', 'D0', 'D1', 'D2', 'D3', 'D4', 'StatisticFormatID',
       'ClimateDivisionsID', 'Region_All',
       ...
       'Region_West Olympic Coast', 'Region_West Ozarks',
       'Region_West Southwest', 'Region_West Upper', 'Region_Western',
       'Region_Western Piedmont', 'Region_Western Plateau',
       'Region_Willamette Valley', 'Region_Wind River',
       'Region_Yellowstone Drainage'],
      dtype='object', length=180)

In [119]:
drought=pd.get_dummies(drought,'Region',drop_first=True)

In [120]:
drought.shape

(409122, 180)

In [121]:

X=drought.drop('D0',axis=1)
y=drought['D0']

In [122]:
X_train, X_val, y_train, y_val = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    )

In [123]:
rf=RandomForestRegressor()

In [124]:
rf.fit(X_train, y_train)

RandomForestRegressor()

In [127]:
rf.score(X_train, y_train)

0.9963730786370225

In [ ]:
drought=pd.read_csv('./drought/dr.csv',decimal=".",sep=',')

In [171]:
df_wes =pd.read_csv('./drought/westerm.csv',decimal=".",sep=',')

In [172]:
df_wes['None'] = df_wes['None'].str.replace(",", "").astype(float)
df_wes['D0'] = df_wes['D0'].str.replace(",", "").astype(float)
df_wes['D1'] = df_wes['D1'].str.replace(",", "").astype(float)
df_wes['D2'] = df_wes['D2'].str.replace(",", "").astype(float)
df_wes['D3'] = df_wes['D3'].str.replace(",", "").astype(float)
df_wes['D4'] = df_wes['D4'].str.replace(",", "").astype(float)

In [173]:
df_wes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MapDate             1044 non-null   int64  
 1   Region              1044 non-null   object 
 2   None                1044 non-null   float64
 3   D0                  1044 non-null   float64
 4   D1                  1044 non-null   float64
 5   D2                  1044 non-null   float64
 6   D3                  1044 non-null   float64
 7   D4                  1044 non-null   float64
 8   ValidStart          1044 non-null   object 
 9   ValidEnd            1044 non-null   object 
 10  StatisticFormatID   1044 non-null   int64  
 11  ClimateDivisionsID  1044 non-null   int64  
dtypes: float64(6), int64(3), object(3)
memory usage: 98.0+ KB


In [174]:
df_wes['no_drought']=df_wes['None']

In [175]:
df_wes['area_Drought']=df_wes['D0']+df_wes['D1']+df_wes['D2']+df_wes['D3']+df_wes['D4']

In [179]:
df_wes.head(50)

,MapDate,Region,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID,ClimateDivisionsID,no_drought,area_Drought,is_in_drought
0,20211214,Western,0.00,0.00,15121.37,3482.23,4908.56,911.29,2021-12-14,2021-12-20,2,2401,0.00,24423.45,1
1,20211207,Western,0.00,0.00,10560.59,6605.00,6335.84,922.02,2021-12-07,2021-12-13,2,2401,0.00,24423.45,1
2,20211130,Western,0.00,0.00,10560.59,8570.23,4370.61,922.02,2021-11-30,2021-12-06,2,2401,0.00,24423.45,1
3,20211123,Western,0.00,0.00,10560.59,8570.23,4370.61,922.02,2021-11-23,2021-11-29,2,2401,0.00,24423.45,1
4,20211116,Western,0.00,0.00,10560.59,8570.23,4370.61,922.02,2021-11-16,2021-11-22,2,2401,0.00,24423.45,1
5,20211109,Western,0.00,0.00,0.00,19130.83,4053.12,1239.51,2021-11-09,2021-11-15,2,2401,0.00,24423.46,1
6,20211102,Western,0.00,0.00,0.00,19130.83,4053.12,1239.51,2021-11-02,2021-11-08,2,2401,0.00,24423.46,1
7,20211026,Western,0.00,0.00,0.00,18295.91,4879.45,1248.10,2021-10-26,2021-11-01,2,2401,0.00,24423.46,1
8,20211019,Western,0.00,0.00,0.00,18343.36,5506.62,573.48,2021-10-19,2021-10-25,2,2401,0.00,24423.46,1
9,20211012,Western,0.00,0.00,0.00,19752.82,4097.16,573.48,2021-10-12,2021-10-18,2,2401,0.00,24423.46,1


In [177]:
df_wes['is_in_drought']=df_wes['no_drought'].map(lambda x: 1 if x==0 else 0)

In [178]:
df_wes['is_in_drought'].value_counts(normalize=True)

0    0.766284
1    0.233716
Name: is_in_drought, dtype: float64